In [12]:
import numpy as np
import pandas as pd
import time
import pickle as pc
import os


In [13]:
def _read_best_hyperapameters():
        """Internal method that reads and extracts the subset of optimal hyperparameters

        The file that is read depends on the provided input parameters to the class
        """

        # Defining the trials file name used to extract the optimal hyperparameters
        trials_file_name = 'DNN_hyperparameters_nl2_datNP_YT2_SF_CW4_1'

        # Reading and extracting the best hyperparameters
        trials = pc.load(open(trials_file_name, "rb"))
        
        best_hyperparameters = format_best_trial(trials.best_trial)

        return best_hyperparameters


In [14]:
test = _read_best_hyperapameters()
test


{'In: Day': 0,
 'In: Exog-1 D': 1,
 'In: Exog-1 D-1': 1,
 'In: Exog-1 D-7': 0,
 'In: Exog-2 D': 1,
 'In: Exog-2 D-1': 1,
 'In: Exog-2 D-7': 0,
 'In: Price D-1': 1,
 'In: Price D-2': 1,
 'In: Price D-3': 0,
 'In: Price D-7': 1,
 'activation': 'relu',
 'batch_normalization': 1,
 'dropout': 0.2813252216984115,
 'init': 'Orthogonal',
 'lambdal1': 0,
 'lr': 0.0008413414653686844,
 'neurons1': 262.0,
 'neurons2': 43.0,
 'reg': None,
 'scaleX': 'Median',
 'scaleY': 'Std',
 'seed': 923.0}

In [15]:
def format_best_trial(best_trial):
    """Function to format the best_trial object to a python dictionary.

    The trials file used to save the hyperparameter optimization contains a lot of information. This
    function extracts from the file the values of the optimal hyperparameters and stores them in a 
    python dictionary. It receives as input a ``trial`` from a ``Trials`` object. 

    Parameters
    ----------
    best_trial : dict
        A trial dictionary as extracted from the ``Trials`` object generated by the 
        :class:`hyperparameter_optimizer`function. 
    
    Returns
    -------
    formatted_hyperparameters : Dict
        Formatted dictionary with the optimal hyperparameters
    """

    # Extracting unformatted hyperparameters
    unformatted_hyperparameters = best_trial['misc']['vals']

    formatted_hyperparameters = {}

    # Removing list format
    for key, val in unformatted_hyperparameters.items():
        if len(val) > 0:
            formatted_hyperparameters[key] = val[0]
        else:
            formatted_hyperparameters[key] = 0

    # Reformatting discrete hyperparameters from integer value to keyword
    for key, val in formatted_hyperparameters.items():
        if key == 'activation':
            hyperparameter_list = ["relu", "softplus", "tanh", 'selu', 'LeakyReLU', 'PReLU', 'sigmoid']
        elif key == 'init':
            hyperparameter_list = ['Orthogonal', 'lecun_uniform', 'glorot_uniform', 'glorot_normal', 
                                   'he_uniform', 'he_normal']
        elif key == 'scaleX':
            hyperparameter_list = ['No', 'Norm', 'Norm1', 'Std', 'Median', 'Invariant']
        elif key == 'scaleY':
            hyperparameter_list = ['No', 'Norm', 'Norm1', 'Std', 'Median', 'Invariant']
        elif key == 'reg':
            hyperparameter_list = [None, 'l1']

        if key in ['activation', 'init', 'scaleX', 'scaleY', 'reg']:
            formatted_hyperparameters[key] = hyperparameter_list[val]

    return formatted_hyperparameters

In [16]:
import sys

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(1, parent_dir)

X_test = pd.read_csv('X_test.csv')
X_train = pd.read_csv('X_train.csv')
y_test = pd.read_csv('y_test.csv')
y_train = pd.read_csv('y_train.csv')
X_train.drop(['day','hour'], axis = 1, inplace = True)
X_test.drop(['day','hour'], axis = 1, inplace = True)
X_train.head()

,wind_generation,solar_generation,demand,index_price,nuclear_generation,reference_price
0,1445.0,0.0,31116.0,13.94,6071.9,53.05
1,8374.0,0.0,20215.0,13.75,6125.5,30.55
2,5618.0,6680.4,31293.0,29.39,5068.5,87.29
3,3395.0,1963.5,29624.0,87.05,7117.2,231.00
4,6386.3,6560.5,31181.8,47.49,6121.4,139.00


In [18]:
"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.

In this example, we optimize the validation accuracy of fashion product recognition using
PyTorch and FashionMNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole FashionMNIST dataset,
we here use a small subset of it.

"""

import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms


DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10


def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)


def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader


def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-07-16 18:05:20,095] A new study created in memory with name: no-name-921b1ef7-047f-493c-b9d6-baa5d0167f18
[W 2024-07-17 08:59:00,074] Trial 0 failed with parameters: {'n_layers': 3, 'n_units_l0': 69, 'dropout_l0': 0.24999842064058098, 'n_units_l1': 32, 'dropout_l1': 0.2667157755024252, 'n_units_l2': 85, 'dropout_l2': 0.4508194249871978, 'optimizer': 'RMSprop', 'lr': 0.000996563822098604} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\ubeda\anaconda3\envs\market_forecast\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\ubeda\AppData\Local\Temp\ipykernel_7108\3877324271.py", line 101, in objective
    for batch_idx, (data, target) in enumerate(valid_loader):
  File "C:\Users\ubeda\AppData\Roaming\Python\Python312\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_frame.py", line 1197, in trace_dispatch
    se

KeyboardInterrupt: 